# AI-Driven Early Prediction of Pulmonary Fibrosis Using Deep Learning
## Classification

In [4]:
import os
import cv2
import pydicom
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm.notebook import tqdm
from skimage import measure, morphology
from pathlib import Path
import json
import time
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 1. CONFIGURATION & DIRECTORY SETUP
# ==========================================
CONFIG = {
    "batch_size": 16,             # Paper suggests powers of 2 (16, 32)
    "epochs": 50,                 # Maximum epochs
    "lr": 2e-5,                   # Standard starting point for Adam
    "image_size": 256,            
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "root_dir": "../input/osic-pulmonary-fibrosis-progression/train",
    "version_dir": "version_2",   # ✅ NEW: Dedicated directory for this run
}

# Create Version 2 Directory Structure
os.makedirs(CONFIG['version_dir'], exist_ok=True)


# Define Log File Path inside version directory
LOG_FILE = os.path.join(CONFIG['version_dir'], 'metrics_log.json')

# ==========================================
# 2. IMPROVED "GROUND TRUTH" GENERATION
# ==========================================
def generate_robust_mask(image):
    """
    Refined function to generate a tighter lung mask boundary.
    """
    # 1. Standardize
    image = image.copy()
    mean = np.mean(image)
    std = np.std(image)
    image = (image - mean) / std
    
    middle = image[100:-100, 100:-100] 
    mean_mid = np.mean(middle)  
    
    # Thresholding to separate lung from chest wall
    thresh_img = np.where(image < mean_mid, 1.0, 0.0)

    # 3. Morphological Cleaning
    eroded = morphology.erosion(thresh_img, np.ones((3,3)))
    
    # ✅ Reduced dilation kernel size from 8x8 to 4x4
    dilation = morphology.dilation(eroded, np.ones((4,4))) 

    # 4. Label regions
    labels = measure.label(dilation)
    regions = measure.regionprops(labels)
    
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0] < CONFIG['image_size']*0.9 and B[3]-B[1] < CONFIG['image_size']*0.9 and prop.area > 500:
            good_labels.append(prop.label)
            
    mask = np.zeros_like(dilation, dtype=np.float32)
    for l in good_labels:
        mask = np.where(labels == l, 1.0, mask)

    # 5. Final cleanup
    # ✅ Reduced dilation kernel size from 6x6 to 3x3
    mask = morphology.dilation(mask, np.ones((3,3))) 
    mask = cv2.GaussianBlur(mask, (5,5), 0) 
    mask = (mask > 0.5).astype(np.float32)
    
    # ✅ Final erosion to actively shrink boundary
    mask = morphology.erosion(mask, np.ones((2,2)))
    
    return mask

def load_scan_and_mask(path):
    try:
        dcm = pydicom.dcmread(path)
        img = dcm.pixel_array.astype(np.float32)
        
        slope = getattr(dcm, 'RescaleSlope', 1)
        intercept = getattr(dcm, 'RescaleIntercept', -1024)
        img = slope * img + intercept
        
        img = cv2.resize(img, (CONFIG['image_size'], CONFIG['image_size'])) 
        
        mask = generate_robust_mask(img)
        
        img_min, img_max = -1000, 400 
        img = np.clip(img, img_min, img_max)
        img = (img - img_min) / (img_max - img_min)
        
    except Exception as e:
        return None, None
        
    return img, mask

# ==========================================
# 3. STANDARD U-NET ARCHITECTURE
# ==========================================
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class StandardUNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        # Encoder
        self.inc = DoubleConv(in_channels, 64)
        self.down1 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(64, 128))
        self.down2 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(128, 256))
        self.down3 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(256, 512))
        self.down4 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(512, 1024))
        
        # Decoder
        self.up1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.conv_up1 = DoubleConv(1024, 512) 
        
        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv_up2 = DoubleConv(512, 256)
        
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv_up3 = DoubleConv(256, 128)
        
        self.up4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv_up4 = DoubleConv(128, 64)
        
        self.outc = nn.Conv2d(64, out_channels, kernel_size=1) 

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4) 
        
        x = self.up1(x5)
        x = torch.cat([x, x4], dim=1) 
        x = self.conv_up1(x)
        
        x = self.up2(x)
        x = torch.cat([x, x3], dim=1)
        x = self.conv_up2(x)
        
        x = self.up3(x)
        x = torch.cat([x, x2], dim=1)
        x = self.conv_up3(x)
        
        x = self.up4(x)
        x = torch.cat([x, x1], dim=1)
        x = self.conv_up4(x)
        
        return torch.sigmoid(self.outc(x))

# ==========================================
# 4. DICE LOSS
# ==========================================
class DiceLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, pred, target):
        smooth = 1e-5
        pred = pred.view(-1)
        target = target.view(-1)
        intersection = (pred * target).sum()
        dice = (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)
        return 1 - dice

# ==========================================
# 5. TRAINING LOOP
# ==========================================

# Setup Data Loaders (Remains the same)
all_patients = sorted([p for p in os.listdir(CONFIG['root_dir']) if os.path.isdir(os.path.join(CONFIG['root_dir'], p))])
all_files = []
for p in all_patients[:50]: 
    p_path = os.path.join(CONFIG['root_dir'], p)
    files = sorted([os.path.join(p_path, f) for f in os.listdir(p_path) if f.endswith('.dcm')])
    idx_start = int(len(files) * 0.3)
    idx_end = int(len(files) * 0.7)
    all_files.extend(files[idx_start:idx_end])

class LungDataset(Dataset):
    def __init__(self, files):
        self.files = files
    def __len__(self): return len(self.files)
    def __getitem__(self, idx):
        img, mask = load_scan_and_mask(self.files[idx])
        if img is None: return self.__getitem__((idx+1)%len(self))
        img = torch.tensor(img).unsqueeze(0).float()
        mask = torch.tensor(mask).unsqueeze(0).float()
        return img, mask

train_files, val_files = train_test_split(all_files, test_size=0.15, random_state=42)
train_loader = DataLoader(LungDataset(train_files), batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
val_loader = DataLoader(LungDataset(val_files), batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)


# 🎯 NEW LR FOR FINE-TUNING
# We assume CONFIG['lr'] is set to 2e-5 before this block is executed
NEW_FINE_TUNING_LR = CONFIG['lr'] 

# Initialize model
model = StandardUNet().to(CONFIG['device'])

# ✅ Tracking variables initialization
BEST_VAL_DICE = -1.0
history = [] 

# --- DYNAMIC LOADING LOGIC ---
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, 'r') as f:
        history = json.load(f)
        
    if history:
        # Find the Absolute Best Checkpoint Path based on F1 Score
        best_entry = max(
            (item for item in history if item.get('checkpoint_path')), 
            key=lambda x: x['val_f1_score'], 
            default=None
        )
        
        if best_entry:
            BEST_MODEL_PATH = best_entry['checkpoint_path']
            BEST_VAL_DICE = best_entry['val_f1_score']

            if os.path.exists(BEST_MODEL_PATH):
                try:
                    state_dict = torch.load(BEST_MODEL_PATH, map_location=CONFIG['device'])
                    model.load_state_dict(state_dict)
                    print(f"✅ Successfully loaded best weights (F1: {BEST_VAL_DICE:.4f}) from {BEST_MODEL_PATH}.")
                except Exception as e:
                    print(f"⚠️ Warning: Error loading best weights ({BEST_MODEL_PATH}): {e}. Starting fresh.")
            else:
                 print(f"⚠️ Warning: Best checkpoint path found in log but file is missing. Starting fresh.")
        
# --- OPTIMIZER INITIALIZATION ---
# Use the low LR for fine-tuning the loaded model
optimizer = optim.Adam(model.parameters(), lr=NEW_FINE_TUNING_LR) 
criterion = DiceLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=5)

print(f"🚀 Starting FINE-TUNING Run... LR: {NEW_FINE_TUNING_LR:.2e}")


for epoch in range(CONFIG['epochs']):
    model.train()
    train_loss = 0
    for img, mask in tqdm(train_loader, leave=False):
        img, mask = img.to(CONFIG['device']), mask.to(CONFIG['device'])
        optimizer.zero_grad()
        pred = model(img)
        loss = criterion(pred, mask)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
    # Validation
    model.eval()
    val_acc_list, val_prec_list, val_rec_list, val_f1_list, val_dice_list = [], [], [], [], []

    with torch.no_grad():
        for img, mask in val_loader:
            img, mask = img.to(CONFIG['device']), mask.to(CONFIG['device'])
            pred = model(img)
            
            # Dice (Loss Metric)
            val_dice_list.append(1 - criterion(pred, mask).item())
            
            # Sklearn Metrics (Threshold 0.5)
            pred_bin = (pred > 0.5).float().cpu().numpy().astype(int).flatten()
            mask_np = mask.cpu().numpy().astype(int).flatten()
            
            val_acc_list.append(accuracy_score(mask_np, pred_bin))
            val_prec_list.append(precision_score(mask_np, pred_bin, zero_division=1))
            val_rec_list.append(recall_score(mask_np, pred_bin, zero_division=1))
            val_f1_list.append(f1_score(mask_np, pred_bin, zero_division=1))
            
    # Aggregate
    avg_dice = np.mean(val_dice_list)
    avg_acc = np.mean(val_acc_list)
    avg_prec = np.mean(val_prec_list)
    avg_rec = np.mean(val_rec_list)
    avg_f1 = np.mean(val_f1_list)
    
    current_lr = optimizer.param_groups[0]['lr']
    scheduler.step(avg_dice)
    
    # ✅ DYNAMIC PATH CONSTRUCTION
    dynamic_filename = f"epoch_{epoch + 1}_f1_{avg_f1:.4f}.pth"
    dynamic_path = os.path.join(CONFIG['version_dir'], dynamic_filename)
    
    epoch_metrics = {
        'epoch': epoch + 1,
        'timestamp': time.time(),
        'learning_rate': current_lr,
        'train_loss': train_loss / len(train_loader),
        'val_accuracy': avg_acc,
        'val_precision': avg_prec,
        'val_recall': avg_rec,
        'val_f1_score': avg_f1,
        'val_dice_score': avg_dice,
        'checkpoint_path': '' 
    }
    
    # ✅ SAVE LOGIC: Save if it beats the best record
    if avg_f1 > BEST_VAL_DICE:
        BEST_VAL_DICE = avg_f1
        torch.save(model.state_dict(), dynamic_path)
        epoch_metrics['checkpoint_path'] = dynamic_path
        print(f"🎉 NEW BEST MODEL SAVED! F1: {avg_f1:.4f} -> {dynamic_path}")
    
    history.append(epoch_metrics)
    with open(LOG_FILE, 'w') as f:
        json.dump(history, f, indent=4)
    
    print(f"\n✅ Epoch {epoch+1} Summary:")
    print(f"   Loss      : {train_loss/len(train_loader):.4f}")
    print(f"   Accuracy  : {avg_acc:.4f}")
    print(f"   Precision : {avg_prec:.4f}")
    print(f"   Recall    : {avg_rec:.4f}")
    print(f"   F1-Score  : {avg_f1:.4f}")
    print("-" * 30)

    if avg_dice > 0.96:
        print("🎉 Target metrics achieved.")
        break

✅ Successfully loaded best weights (F1: 0.8990) from version_2/epoch_7_f1_0.8990.pth.
🚀 Starting FINE-TUNING Run... LR: 2.00e-05


  0%|          | 0/220 [00:00<?, ?it/s]

🎉 NEW BEST MODEL SAVED! F1: 0.8992 -> version_2/epoch_1_f1_0.8992.pth

✅ Epoch 1 Summary:
   Loss      : 0.1133
   Accuracy  : 0.9762
   Precision : 0.9117
   Recall    : 0.8913
   F1-Score  : 0.8992
------------------------------


  0%|          | 0/220 [00:00<?, ?it/s]

🎉 NEW BEST MODEL SAVED! F1: 0.8992 -> version_2/epoch_2_f1_0.8992.pth

✅ Epoch 2 Summary:
   Loss      : 0.1120
   Accuracy  : 0.9762
   Precision : 0.9114
   Recall    : 0.8915
   F1-Score  : 0.8992
------------------------------


...

🎉 NEW BEST MODEL SAVED! F1: 0.9014 -> version_2/epoch_15_f1_0.9014.pth

✅ Epoch 15 Summary:
   Loss      : 0.1084
   Accuracy  : 0.9768
   Precision : 0.9151
   Recall    : 0.8923
   F1-Score  : 0.9014
------------------------------


  0%|          | 0/220 [00:00<?, ?it/s]

🎉 NEW BEST MODEL SAVED! F1: 0.9021 -> version_2/epoch_16_f1_0.9021.pth

✅ Epoch 16 Summary:
   Loss      : 0.1102
   Accuracy  : 0.9769
   Precision : 0.9125
   Recall    : 0.8962
   F1-Score  : 0.9021
------------------------------


  0%|          | 0/220 [00:00<?, ?it/s]

KeyboardInterrupt: 